# testcell

In [ ]:
#| hide
import testcell

**TL;DR**: `%%testcell` prevents your testing cells from affecting the global namespace.

The Python cell magic `%%testcell` executes a cell without *polluting* the notebook's global variables. This is useful whenever you want to test your code without having any of the local variables escape that cell.

What's happening under the hood is that your cell code, before being executed, is wrapped in a temporary function that will be deleted after execution. To give you the feeling of "seamless integration" the last line is optionally wrapped with a `display` statement if needed.

**WARNING:** this don't protect you from *the side effects of your code* like deleting a file or mutating the state of a global variable.

## Install

```sh
pip install testcell
```

## How to use

just import it with `import testcell` and then use the `%%testcell` cell magic.

```python
%%testcell
a = "'a' is not polluting global scope"
a
```

In [ ]:
%%testcell
#| echo: false
a = "'a' is not polluting global scope"
a

"'a' is not polluting global scope"

In [ ]:
assert 'a' not in locals()

What is happening under the hood is that `%%testcell` wraps your cell's code with a function, execute it and then deletes it. Adding the `verbose` keywork will print which code will be executed.

```python
%%testcell verbose
a = "'a' is not polluting global scope"
a
```

In [ ]:
%%testcell verbose
#| echo: false
a = "'a' is not polluting global scope"
a


### BEGIN
def _test_cell_():
	#| echo: false
	a = "'a' is not polluting global scope"
	display(a)
try:
	_test_cell_()
finally:
	del _test_cell_
### END


"'a' is not polluting global scope"

If you're just interested in seeing what will be executed, but actually not executing it, you ca use `dryrun` option:

```python
%%testcell dryrun
a = "'a' is not polluting global scope"
a
```

In [ ]:
%%testcell dryrun
#| echo: false
a = "'a' is not polluting global scope"
a


### BEGIN
def _test_cell_():
	#| echo: false
	a = "'a' is not polluting global scope"
	display(a)
try:
	_test_cell_()
finally:
	del _test_cell_
### END


On top of *wrapping* your cell's code, it will optinally add a `display(...)` call on your last cell's row, trying to emulate what a real cell usually does. If you explicitly add a semicolon `;` in the end, this output will be suppressed.

```python
%%testcell verbose
a = "'a' is not polluting global scope"
a;
```

In [ ]:
%%testcell verbose
#| echo: false
a = "'a' is not polluting global scope"
a;


### BEGIN
def _test_cell_():
	#| echo: false
	a = "'a' is not polluting global scope"
	a;
try:
	_test_cell_()
finally:
	del _test_cell_
### END


There are more cases where `display(...)` is avoided:
* `display`: if this is already present on the last line.
* `print`: even in this case no print is preserved and no other display call is added.
* `# comment...#`: these are preserved 

```python
%%testcell verbose
a = "'a' is not polluting global scope"
print(a)
```

In [ ]:
%%testcell verbose
#| echo: false
a = "'a' is not polluting global scope"
print(a)


### BEGIN
def _test_cell_():
	#| echo: false
	a = "'a' is not polluting global scope"
	print(a)
try:
	_test_cell_()
finally:
	del _test_cell_
### END
'a' is not polluting global scope


**IMPORTANT**: this is *just wrapping your cell*  and so it's still running on your main kernel. If you modify variables that has been created outside of this cell (aka: if you have side effects) this will not protect you.

## TODO:

+ Install as a plugin to enable it by default like other cell's magic.
+ Eval possibility to run code on a new kernel to reduce even more possible side effects.